In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow.keras.backend as K
tf.compat.v1.enable_eager_execution(
    config=None, device_policy=None, execution_mode=None
)
print("Eager execution: {}".format(tf.executing_eagerly()))

C:\Users\Aymen\.conda\envs\p36workshop\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Aymen\.conda\envs\p36workshop\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Aymen\.conda\envs\p36workshop\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Aymen\.conda\envs\p36workshop\lib\site-packages\tensorf

Eager execution: True


BRU :                  https://github.com/marek-kan/Bionodal-root-units/blob/master/BRU_ConV.ipynb <br>
Sweep hyperparameters :https://docs.wandb.ai/guides/sweeps/quickstart <br>
Sweep example:         https://github.com/wandb/examples

In [2]:
!wandb login ab631efc36e2c87f5f54d82b5cdbd6c501d5221f

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Aymen/.netrc


In [21]:
@tf.custom_gradient
def O2RU(x):
    r=2
    g =  r*(r**2. * K.abs(x) + 1.)**((1.-r)/r)
    def grad(dy):
        return dy * g #dy = 1 
    return K.sign(x)*((r**2. * K.abs(x) + 1.)**(1./r) - 1.), grad

@tf.custom_gradient
def O3RU(x):
    r=3
    g =  r*(r**2. * K.abs(x) + 1.)**((1.-r)/r)
    def grad(dy):
        return dy * g #dy = 1 
    return K.sign(x)*((r**2. * K.abs(x) + 1.)**(1./r) - 1), grad


# if you want to check diff in 0, automatic diff by TF gives 0 but it should be r
with tf.GradientTape() as g:
    x = tf.constant([0.0],dtype=tf.float32)
    g.watch(x)
    y = O3RU(x)
dy_dx = g.gradient(y, x)
print(dy_dx)

@tf.custom_gradient
def E1RU(x):
    r = 1
    gradient = tf.where(x >= 0, r*(r**2. * x + 1.)**((1.-r)/r), r*K.exp(r*x))
    def grad(dy):
        return dy * gradient
    return tf.where(x >= 0, (r**2. * x + 1.)**(1./r) - 1./r, K.exp(r*x) - 1/r), grad

@tf.custom_gradient
def E2RU(x):
    r = 2
    gradient = tf.where(x >= 0, r*(r**2. * x + 1.)**((1.-r)/r), r*K.exp(r*x))
    def grad(dy):
        return dy * gradient
    return tf.where(x >= 0, (r**2. * x + 1.)**(1./r) - 1./r, K.exp(r*x) - 1/r), grad

@tf.custom_gradient
def E3RU(x):
    r = 3
    gradient = tf.where(x >= 0, r*(r**2 * x + 1.)**((1.-r)/r), r*K.exp(r*x))
    def grad(dy):
        return dy * gradient
    return tf.where(x >= 0, (r**2. * x + 1.)**(1./r) - 1./r, K.exp(r*x) - 1/r), grad

with tf.GradientTape() as g:
    x = tf.constant([-5.0,0.0,5.0,10.0,100.0],dtype=tf.float32)
    g.watch(x)
    y = E2RU(x)
dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor([3.], shape=(1,), dtype=float32)
tf.Tensor([9.0799862e-05 2.0000000e+00 4.3643579e-01 3.1234753e-01 9.9875234e-02], shape=(5,), dtype=float32)


In [22]:
def custom_relu(x):
    return K.maximum(0.0,x)
model = tf.keras.models.Sequential([
     tf.keras.layers.Flatten(input_shape=(128,128)),
     tf.keras.layers.Dense(512),
     tf.keras.layers.Lambda(custom_relu),
     tf.keras.layers.Dense(5, activation = 'softmax')
])

In [23]:
df=pd.read_csv("../data/GBPAUD_step0.csv")
size = int(len(df['Close'])*0.8)
df_new_train = pd.DataFrame(index=df['Close'].iloc[:size].index)
df_new_test = pd.DataFrame(index=df['Close'].iloc[size:].index)
df_new_train['Close'], df_new_test['Close'] = df['Close'].iloc[:size], df['Close'].iloc[size:]

In [24]:
train_dataset = tf.data.Dataset.from_tensor_slices((df_new_train['Close']))
test_dataset = tf.data.Dataset.from_tensor_slices(df_new_test['Close'])

In [25]:
train_dataset

<DatasetV1Adapter shapes: (), types: tf.float64>

In [33]:
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 100
features=list() 
labels=list()
#features, labels = next(iter(train_dataset))
for i,element in enumerate(train_dataset):
    labels.append(float(i))
    features.append(element)

In [34]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    y_ = model(x, training=training)
    return loss_object(y_true=y, y_pred=y_)

In [35]:
def grad(model, inputs, targets=None):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs,_, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [36]:
tf.compat.v1.random.set_random_seed(0)
shape = (32,32,3)
model_bru = tf.keras.Sequential([
                                 tf.keras.layers.Conv1D(filters=96,kernel_size=(3,3), strides=1, padding='same', 
                                                        activation=E3RU, input_shape=shape),
                                 tf.keras.layers.Conv1D(filters=96,kernel_size=(2,2), strides=1, padding='valid', 
                                                        activation=O3RU),
                                 tf.keras.layers.MaxPool1D(pool_size=(3,3), strides=2, padding='valid'),
                                 tf.keras.layers.Lambda(E1RU),
                                 tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same',
                                                        activation=E2RU),
                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same',
                                                        activation=O2RU),
                                 tf.keras.layers.MaxPool1D(pool_size=(3,3), strides=2, padding='valid'),
                                 tf.keras.layers.Lambda(E1RU),
                                 tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', 
                                                        activation=E2RU),
                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(1,1), strides=1, padding='same',
                                                        activation=O2RU),
                                 tf.keras.layers.Conv1D(filters=10,kernel_size=(1,1), strides=1, padding='same', 
                                                        activation=E1RU),
                                 tf.keras.layers.AveragePooling2D(pool_size=(6,6), strides=1, padding='same'),
                                 tf.keras.layers.Lambda(E1RU),
                                 #tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(10, activation='softmax')
])

model_elu = tf.keras.Sequential([
                                 tf.keras.layers.Conv1D(filters=96,kernel_size=(3,3), strides=1, padding='same', activation='elu', input_shape=shape),
                                 tf.keras.layers.Conv1D(filters=96,kernel_size=(2,2), strides=1, padding='valid', activation='elu'),
                                 tf.keras.layers.MaxPool1D(pool_size=(3,3), strides=2, padding='valid'),
                                 tf.keras.layers.Lambda(tf.nn.elu),
                                 tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', activation='elu'),
                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', activation='elu'),
                                 tf.keras.layers.MaxPool1D(pool_size=(3,3), strides=2, padding='valid'),
                                 tf.keras.layers.Lambda(tf.nn.elu),
                                 tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', activation='elu'),
                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(1,1), strides=1, padding='same', activation='elu'),
                                 tf.keras.layers.Conv1D(filters=10,kernel_size=(1,1), strides=1, padding='same', activation='elu'),
                                 tf.keras.layers.AveragePooling2D(pool_size=(6,6), strides=1, padding='same'),
                                 tf.keras.layers.Lambda(tf.nn.elu),
                                 #tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(10, activation='softmax')
])

model_relu = tf.keras.Sequential([
                                 tf.keras.layers.Conv1D(filters=96,kernel_size=(3,3), strides=1, padding='same', activation='relu', input_shape=shape),
                                 tf.keras.layers.Conv1D(filters=96,kernel_size=(2,2), strides=1, padding='valid', activation='relu'),
                                 tf.keras.layers.MaxPool1D(pool_size=(3,3), strides=2, padding='valid'),
                                 tf.keras.layers.Lambda(tf.nn.relu),
                                 tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', activation='relu'),
                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', activation='relu'),
                                 tf.keras.layers.MaxPool1D(pool_size=(3,3), strides=2, padding='valid'),
                                 tf.keras.layers.Lambda(tf.nn.relu),
                                 tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', activation='relu'),
                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(1,1), strides=1, padding='same', activation='relu'),
                                 tf.keras.layers.Conv1D(filters=10,kernel_size=(1,1), strides=1, padding='same', activation='relu'),
                                 tf.keras.layers.AveragePooling1D(pool_size=(6,6), strides=1, padding='same'),
                                 tf.keras.layers.Lambda(tf.nn.relu),
                                 #tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(10, activation='softmax')
])

model_leaky_relu = tf.keras.Sequential([
                                 tf.keras.layers.Conv1D(filters=96,kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.leaky_relu, input_shape=shape),
                                 tf.keras.layers.Conv1D(filters=96,kernel_size=(2,2), strides=1, padding='valid', activation=tf.nn.leaky_relu),
                                 tf.keras.layers.MaxPool1D(pool_size=(3,3), strides=2, padding='valid'),
                                 tf.keras.layers.Lambda(tf.nn.leaky_relu),
                                 tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.leaky_relu),
                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.leaky_relu),
                                 tf.keras.layers.MaxPool1D(pool_size=(3,3), strides=2, padding='valid'),
                                 tf.keras.layers.Lambda(tf.nn.leaky_relu),
                                 tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.leaky_relu),
                                 tf.keras.layers.Conv1D(filters=192,kernel_size=(1,1), strides=1, padding='same', activation=tf.nn.leaky_relu),
                                 tf.keras.layers.Conv1D(filters=10,kernel_size=(1,1), strides=1, padding='same', activation=tf.nn.leaky_relu),
                                 tf.keras.layers.AveragePooling1D(pool_size=(6,6), strides=1, padding='same'),
                                 tf.keras.layers.Lambda(tf.nn.leaky_relu),
                                 #tf.keras.layers.Dropout(0.5),

                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(10, activation='softmax')
])

In [37]:
optimizer = tf.keras.optimizers.SGD(0.01)

loss_value, grads = grad(model_bru, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),loss_value.numpy()))

ValueError: Incompatible type conversion requested to type 'float64' for variable of type 'float32'

In [38]:
import time

def train(model, data, num_epochs):
    train_loss_results = []
    train_accuracy_results = []

    epoch = 0
    for epoch in range(num_epochs):
        start = time.time()
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    # Training loop - using batches of 64
    for x, y in data:
        # Optimize the model
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Track progress
        epoch_loss_avg(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        epoch_accuracy(y, model(x, training=True))

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    
    #if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))
    print ('Time taken for epoch {} sec\n'.format(time.time() - start))
    epoch +=1
    return train_loss_results, train_accuracy_results

In [39]:
train_epoch = 40
l_bru, a_bru = train(model_bru, train_dataset, train_epoch)

TypeError: Cannot iterate over a scalar tensor.

In [ ]:
def build_model(input_shape):
    """Generates CNN model

    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv1D(32, 
                                  (3, 3),
                                  kernel_initializer='he_normal',
                                  #activation='relu',
                                  input_shape=(input_shape))
                                 )
    model.add(keras.layers.MaxPooling1D((3, 3),
                                        strides=(2, 2), 
                                        padding='full'),
                                        )
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, 
                                  (3, 3), 
                                  activation='relu'),
              kernel_regularizer="l1_l2",
              bias_regularizer="l2",
                activity_regularizer="l2")
    model.add(keras.layers.MaxPooling2D((3, 3),
                                        strides=(2, 2), 
                                        padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'),
              kernel_regularizer="l1_l2",
              bias_regularizer="l2",
                activity_regularizer="l2")
    model.add(keras.layers.Dropout(0.18))
    
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.12))

    # output layer
    model.add(keras.layers.Dense(13, activation='softmax'))

    return model


def predict(model, X, y):
    """Predict a single sample using the trained model

    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1)

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))